# Study-02-AntWorld++ Experiment - 1
Details: [Notion Link](https://www.notion.so/AntWorld-Experiment-1-6c1c887df98b44bf8aaf36873a02d86a)

In [1]:
import itertools
import copy
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as P
import gym
from gym import spaces
from tqdm import tqdm
import seaborn as sns

import torch
from torch.utils.data import DataLoader
from torch.nn import functional as F
from pytorch_lightning.core.lightning import LightningModule
from pytorch_lightning import Trainer

import pyRC.datasets.nordland as Nordland
import pyRC.analyse.utils as utA
import pyRC.datasets.utils as utD
import pyRC.learn.utils as utL
from pyRC.network import ESN_NA
from pyRC.environments import AntWorld

# Plotting settings
sns.set_context("notebook", font_scale = 1.5)
sns.set_style("dark")
sns.set_palette("deep", 12)
from ipywidgets import interact, interactive, fixed, interact_manual
cmap='turbo'

## Load Environment

In [2]:
# Ant environment
env = AntWorld('AntWorld-Gym-02')

/home/anil/.virtualenv/pyRC2-vEnv/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


### Prepare environment for training

In [3]:
# Img = np.load('AntWorld-Gym-01.npz')['Img'] # Generic Environment
Img = env.Img
print(Img.shape)
height, width = Img.shape[1:]

(8000, 10, 36)


In [4]:
Imgs, Lbls = [], []
for x in np.arange(0, 2, 0.05): # Label: 1
    Imgs.append(env.getImage([x,0],0))
    Lbls.append(1) 
for x in np.arange(2, 4, 0.05): # Label: 0
    Imgs.append(env.getImage([x,0],0))
    Lbls.append(0)

nImg  = len(Imgs)
Image = torch.Tensor(Imgs).unsqueeze(-1) # should be of size [nImg, h, w, 1]
Label = [1] * int(nImg/2) + [0] * int(nImg/2) # half 1, half 0 

dataSet    = utD.ImageDataset(Image, Label)
dataLoader = DataLoader(dataSet,batch_size=1, shuffle=False)

## PyTorch Lignthning Module for Experiment

In [5]:
class Experiment(LightningModule):
    def __init__(self, trainData):
        super().__init__()
        self.hparams     = {**config['hyperparameters'], **config['experimentParameters'], **config['modelParameters'], **config['experimentDetails']} # unzip nested dict
        self.trainData   = trainData
        self.RC          = ESN_NA(self.hparams)
        self.GroundTruth = torch.Tensor(dataLoader.dataset.labels)
        
        
        self.FC1         = torch.nn.Linear(self.hparams['nOutput'],100)
        self.FC2         = torch.nn.Linear(100,10)
        self.FC3         = torch.nn.Linear(10, 1)
        self.NET         = torch.nn.Sequential(self.RC, self.FC1, self.FC2, self.FC3)
#         self.params      = self.RC.params + [{'params': x.parameters(), 'lr':self.hparams['learningRate']} for x in [self.FC1, self.FC2, self.FC3]]
        
    def forward(self, x):
        return self.NET(x)
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.NET.parameters())
    
    def train_dataloader(self):
        return self.trainData
    
    def test_dataloader(self):
        return self.trainData
    
    def training_step(self, batch, batch_idx):
        if batch_idx == 0: self.RC.reset() # At the beginning of each epoch, reset the model! 
        
        x, y  = batch
        y_hat = self(x)
        loss  = F.cross_entropy(y_hat, y)
        logs_loss = {'train_loss': loss}
        return {'loss': loss, 'log': logs_loss}
    
    def test_step(self, batch, batch_idx): 
        x, y    = batch
        y_hat   = self(x)
        tol_acc = (torch.abs(self.GroundTruth[batch_idx] - torch.argmax(y_hat))<self.hparams['tolerance']) # Accuracy
        return {'tol_acc': tol_acc, 'y_pred': y_hat}

    def test_epoch_end(self, outputs):
        acc  = torch.stack([x['tol_acc'] for x in outputs]).float().mean()*100 # percentage
        logs = {'test_acc': acc}
        return logs

## Create Model from config

In [6]:
config = utL.getConfig('config.yaml')
config['experimentDetails']['labelFreq'] = nImg//2
config['modelParameters']['nOutput']     = 1000

In [7]:
expDict = {
            'gpus'                     : 1,
            'profiler'                 : False,
            'log_save_interval'        : 10000,
            'progress_bar_refresh_rate': 1000,
            'row_log_interval'         : 10000,
            'max_epochs'               : 10,
            }

In [8]:
# exp     = Experiment(dataLoader)

In [9]:
# x = next(iter(dataLoader))[0]
# # exp(x.to('cuda'))
# # exp.params
# trainer = Trainer(**expDict)
# trainer.fit(exp); 

In [10]:
# exp     = Experiment(dataLoader)
# trainer = Trainer(**expDict)
# trainer.fit(exp); 
# trainer.test(ckpt_path=None);

## Debug: Create ESN-MLP

In [11]:
hparams     = {**config['hyperparameters'], **config['experimentParameters'], **config['modelParameters'], **config['experimentDetails']} # unzip nested dict
RC          = ESN_NA(hparams)
FC1         = torch.nn.Linear(hparams['nOutput'],100)
FC2         = torch.nn.Linear(100,10)
FC3         = torch.nn.Linear(10, 1)
NET         = torch.nn.Sequential(RC,FC1,FC2,FC3)

In [12]:
x = next(iter(dataLoader))[0].to('cuda')

In [13]:
RC.reset()
y=RC(x)
FC1(y)

tensor([[-0.0249,  0.0114,  0.0204, -0.0009, -0.0258,  0.0279,  0.0293, -0.0268,
         -0.0074,  0.0166, -0.0013,  0.0029, -0.0303,  0.0183,  0.0033, -0.0309,
         -0.0255,  0.0167,  0.0048,  0.0182,  0.0262, -0.0136, -0.0199, -0.0132,
         -0.0260,  0.0276, -0.0269,  0.0205,  0.0140,  0.0019,  0.0114, -0.0254,
         -0.0209, -0.0250,  0.0182, -0.0302,  0.0202, -0.0119,  0.0025, -0.0186,
         -0.0185,  0.0164, -0.0051,  0.0057, -0.0062,  0.0126, -0.0257, -0.0125,
         -0.0039,  0.0271,  0.0221,  0.0297, -0.0076,  0.0142,  0.0048, -0.0192,
         -0.0205,  0.0299,  0.0283, -0.0238,  0.0044,  0.0266,  0.0271, -0.0286,
          0.0006,  0.0311,  0.0124, -0.0176,  0.0067,  0.0141, -0.0148, -0.0213,
         -0.0024, -0.0016, -0.0308,  0.0087, -0.0170, -0.0011,  0.0197,  0.0215,
         -0.0006,  0.0296,  0.0301,  0.0237,  0.0180,  0.0061,  0.0131,  0.0261,
         -0.0298, -0.0041, -0.0171,  0.0113,  0.0238,  0.0078, -0.0059,  0.0056,
         -0.0136, -0.0010, -

In [15]:
y.cpu()

RuntimeError: CUDA error: an illegal memory access was encountered